<a href="https://colab.research.google.com/github/beyg1/Q4/blob/main/Quiz%20Practice/Tool_Use_Behaviour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.4/161.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, set_tracing_export_api_key, trace
from openai import AsyncOpenAI
from google.colab  import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
set_tracing_export_api_key(OPENAI_API_KEY)

Client = AsyncOpenAI(
    api_key = GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash-lite",
    openai_client= Client
)                                                                      #Setting up the environment

#Synchronus & Asynchronus Behaviour

In [14]:
import asyncio
from agents import function_tool

@function_tool
def get_weather(location: str) -> str:          #Async Runner & Agent with sync function as tool
  return f"The weather in {location} is Sunny"

async def main():
  agent = Agent(
      name = "Assistant",
      instructions = "Concise Assistant",
      model = model,
      tools = [get_weather]
  )

  with trace("Tool Use"):
    res = await Runner.run(agent,"What's weather like in Karanchi?")
    print(res.final_output)

if __name__ == "__main__":
  asyncio.run(main())

The weather in Karanchi is Sunny.


In [15]:
import asyncio
from agents import function_tool

@function_tool
def get_weather(location: str) -> str:
  return f"The weather in {location} is Sunny"

agent = Agent(
      name = "Assistant",
      instructions = "Concise Assistant",
      model = model,
      tools = [get_weather]
 )

async def main():

  with trace("Tool Use"):
    res = await Runner.run(agent,"What's weather like in Karanchi?")
    print(res.final_output)           #Async Runner  with synchronus Agent & function  as a tool

if __name__ == "__main__":
  asyncio.run(main())

The weather in Karanchi is Sunny.


In [16]:
import asyncio
from agents import function_tool

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:          #Async Runner & Agent & function as tool
   return  f"The weather in {location} is Sunny"

  agent = Agent(
      name = "Assistant",
      instructions = "Concise Assistant",
      model = model,
      tools = [get_weather]
  )

  with trace("Tool Use"):
    res = await Runner.run(agent,"What's weather like in Karanchi?")
    print(res.final_output)

if __name__ == "__main__":
  asyncio.run(main())

The weather in Karanchi is Sunny.


In [19]:
import asyncio
from agents import function_tool

async def main():
  @function_tool
  async def  get_weather(location: str) -> str:          #Async  Agent & function as tool with sync runner
   return  f"The weather in {location} is Sunny"

  agent = Agent(
      name = "Assistant",
      instructions = "Concise Assistant",
      model = model,
      tools = [get_weather]
  )

if __name__ == "__main__":
  asyncio.run(main())

with trace("Tool Use"):
 res =  Runner.run_sync(agent,"What's weather like in Karanchi?")
 print(res.final_output)

The weather in Karanchi is Sunny.
